Fails experiments

In [ ]:
from pathlib import Path

In [ ]:
current_directory = Path.cwd().joinpath("datasets", "today_news")

In [ ]:
today_news = current_directory.joinpath("2024-10-15-news.csv")

In [ ]:
import pandas as pd

In [ ]:
news_df = pd.read_csv(today_news)

In [ ]:
model_path = Path.cwd().joinpath("models")

In [ ]:
embedding_model_id = "dunzhang/stella_en_400M_v5"

In [ ]:
embedding_model_path = model_path.joinpath(embedding_model_id)

In [ ]:

transformer_kwargs = {"model_name_or_path": embedding_model_path.__str__(),
                      "trust_remote_code": True,
                      "device": "cpu",
                      "config_kwargs": {"use_memory_efficient_attention": False,
                                        "unpad_inputs": False},
                      "cache_folder": model_path}

In [ ]:
transformer_kwargs

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
sentence_transformer_model = SentenceTransformer(
    **transformer_kwargs)

In [ ]:
title_embeddings = sentence_transformer_model.encode(news_df["title"].values.tolist(), show_progress_bar=True)

In [ ]:
title_embeddings.shape

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

In [ ]:
def find_best_estimator(X):
    K_mean_estimators = [
        (f"KMeans_{i}", KMeans(n_clusters=i, random_state=42)) for i in range(2, X.shape[0])]
    pca_estimator = PCA(n_components=X.shape[0])

    best_estimator = None
    best_metric = float("-inf")
    intertias  = []
    for estimator_name, estimator in K_mean_estimators:
        transformed_X = pca_estimator.fit_transform(X)
        estimator.fit(transformed_X)
        labels = estimator.labels_
        score = silhouette_score(
            transformed_X, labels)
        if score > best_metric:
            best_metric = score
            best_estimator = estimator
        intertias.append(estimator.inertia_)
    return best_estimator, intertias

In [ ]:
best_estimator, intertias = find_best_estimator(title_embeddings)

In [ ]:
best_estimator.inertia_

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(2, title_embeddings.shape[0]), intertias)

We can see that the shiloulette score does never reach 1, the maximum I can get is 0.12, which indicate that all my cluster a overlaping and k mean does not perform very well here. Let me a hierachical clustering approach.

In [ ]:
from sklearn.metrics import silhouette_samples

In [ ]:
sample_silhouette_values = silhouette_samples(title_embeddings, best_estimator.labels_)

HDBSCan

In [ ]:
import hdbscan

In [ ]:
clustered = hdbscan.HDBSCAN(min_cluster_size=6, min_samples=3, cluster_selection_epsilon=0.75).fit(title_embeddings)

In [ ]:
clustered.labels_

Hiearchical clustering

In [ ]:
from sklearn.neighbors import kneighbors_graph

connectivity = kneighbors_graph(title_embeddings, n_neighbors=10, include_self=False, metric="cosine")

In [ ]:
import time

from sklearn.cluster import AgglomerativeClustering

In [ ]:
similarities = sentence_transformer_model.similarity(title_embeddings, title_embeddings)

In [ ]:
cosine_distance = 1 - similarities

In [ ]:
print("Compute structured hierarchical clustering...")
st = time.time()
agglomerative = AgglomerativeClustering(connectivity=connectivity,
                                        linkage="complete",
                                        metric="cosine",
                                        n_clusters=6)

agglomerative.fit(title_embeddings)
mean_shilouette_score = silhouette_score(title_embeddings, agglomerative.labels_)
elapsed_time = time.time() - st
label = agglomerative.labels_

In [ ]:
label

In [ ]:
news_df["labels_agglomerative"] = label

In [ ]:
def analyse_embeddings(dataframe, embeddings, index, label_column="labels"):
    """ take an matrix of embeddings, and the labels.
    for each label compute the cosine similarity of document with that label.
    """
    document_in_index = dataframe.query(f"{label_column} == {index}")
    with pd.option_context('display.max_colwidth', None):
        display(document_in_index.title)
    document_index = document_in_index.index
    vectors = embeddings[document_index]
    return sentence_transformer_model.similarity(vectors,  vectors)

In [ ]:
analyse_embeddings(news_df, title_embeddings, 0, "labels_agglomerative")

In [ ]:
news_df.query("labels_agglomerative == 0")

In [ ]:
import numpy as np

In [ ]:
pca = PCA(n_components=2)
pca_embeddings = pca.fit_transform(title_embeddings)

In [ ]:
import matplotlib.pyplot as plt

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.set_position([0, 0, 0.95, 1])
for l in np.unique(label):
    ax1.scatter(
        pca_embeddings[label == l, 0],
        pca_embeddings[label == l, 1],
        color=plt.cm.jet(float(l) / np.max(label + 1)),
        s=20,
        edgecolor="k",
    )
_ = fig1.suptitle(
    f"Without connectivity constraints (time {elapsed_time:.2f}s)")